In [1]:
# -------------------------------------------------------------------------------------------------------- #
# Intro de siempre

PATHFunciones = "/home/isabel/Dropbox/git-repos/Spinal-Cord-HDMEA";
PATHBRWs = "/run/media/isabel/Data/Medula/28-02-2022/BRWs";

push!( LOAD_PATH, PATHFunciones );
using JuliaTools

# -------------------------------------------------------------------------------------------------------- #

In [ ]:
# -------------------------------------------------------------------------------------------------------- #

using JLD

local temp = readdir( dirname( PATHBRWs ), join = true );
WorkDirs = temp[ readdir( dirname( PATHBRWs ) ) .!= basename( PATHBRWs ) ];


p = 0.90;
@time for i = 1:length( WorkDirs )
    
    Variables = load( joinpath( WorkDirs[ i ], "Info", "variablesBRW.jld" ) )[ "Variables" ];
    PATHS = load( joinpath( WorkDirs[ i ], "Info", "Paths.jld" ) )[ "PATHS" ];
    PATHVoltage = joinpath( WorkDirs[ i ], "Voltage" );
    FILESVoltage = searchdir( PATHVoltage, "jld" ); 

    nChs = length( Variables[ "Layout" ] ); # total number of channels (...4096)

    sparsity_matrix = zeros( length( FILESVoltage ), 3 ); # Array n segments rows x 3 columns
    variabilidades = zeros( nChs, length( FILESVoltage ) ); # Array 4096 rows x n segments columns
    channels = 1:nChs;

@time for j = 1:length( FILESVoltage );
        
        FILEVoltage = FILESVoltage[ j ];
        data = Float64.( load( FILEVoltage )[ "data" ] ); 

        V = zeros( Int, nChs ); # preallocation
        [ V[ k ] = length( unique( data[ k, : ] ) ) for k in channels ];

        xmean, Δx, C1, C2 = MeanΔxCI( V, p ); AboveCI = findall( V .>= C1 ); 
        grupos, loose = Get_Groups( AboveCI );
        sparsity_matrix[ j, : ] = [ 
            Density( grupos ), length( AboveCI ), Sparsity( length( grupos ), nChs ) ];

        # V vector of each segment into an array
        variabilidades[ :, j ] = V;
        println( "file: ", lpad( j, 2, "0" ), " done" );
        
    end
    
    local temp = joinpath( WorkDirs[ i ], "Info", "Reparations.jld" );
    save( temp, "Sparsity", sparsity_matrix, "Variability", variabilidades );
    println( "file: ", lpad( i, 2, "0" ), "/", length( WorkDirs ), " done" );
    
end
# -------------------------------------------------------------------------------------------------------- #